<a href="https://colab.research.google.com/github/KevinTheRainmaker/MLOps/blob/main/MLOps_07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#데이터 전처리 using TFT

In [ ]:
!pip install -q tensorflow-transform

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
multiprocess 0.70.12.2 requires dill>=0.3.4, but you have dill 0.3.1.1 which is incompatible.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.0 which is incompatible.


In [ ]:
import tensorflow_transform as tft

def preprocessing_fn(inputs):
  x = inputs['x']
  x_normalized = tft.scale_to_0_1(x)

  return {
      'x_xf': x_normalized
  }

In [ ]:
import tensorflow as tf

def preprocessing_fn(raw_image):
  raw_image = tf.reshape(raw_image, [-1])

  # JPEG Decoding
  img_rgb = tf.io.decode_jpeg(raw_image, channels=3)

  # RGB to Gray
  img_gray = tf.image.rgb_to_grayscale(img_rgb)
  img = tf.image.convert_image_dtype(img_gray, tf.float32)

  # Re-sizing the image to 300*300
  resized_img = tf.image.resize_with_pad(
      img,
      target_height=300,
      target_width=300
  )

  img_grayscale = tf.image.rgb_to_grayscale(resized_img)

  return tf.reshape(img_grayscale, [-1, 300, 300, 1])